In [26]:
import pandas as pd
import numpy as np

In [27]:
data = pd.read_csv("Ecommerce Customers.csv")

data_num = data[["Avg. Session Length","Time on App","Time on Website","Length of Membership","Yearly Amount Spent"]]


In [28]:
#data.describe(include='all') #inkludiert auch kategorielle Variabeln 

In [29]:
#data_num.head(5)

In [30]:
#data.isnull().sum() #Überprüfung auf fehlende Werte je Spalte 

Adress analysis

In [31]:
list_shortcut_states = ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY",]
state_list = []

for i in data["Address"]:
    if "Box" in i:
        index_to_drop = data[data["Address"] == i].index.values 
        data.drop(index_to_drop, inplace = True)
    else:
        state = i.split(",")[-1].split()[0]

        if state in list_shortcut_states:
            state_list.append(state)
        else:
            index_to_drop = data[data["Address"] == i].index.values 
            data.drop(index_to_drop, inplace = True)


data.drop(["Email","Avatar","Address"], inplace = True, axis = 1)
data.insert(5, "State", state_list)

data_mixed = data
data_mixed.index = range(len(data_real.index))

#print(data_real.info())
print(data_mixed.State.value_counts())

DE    13
MO    13
SC    13
OR    12
VT    12
MS    11
KS    11
FL    11
MN    11
NC    10
NJ    10
MI     9
HI     9
WV     9
ME     9
ND     9
AL     9
AZ     9
NY     8
PA     8
KY     8
TX     8
IL     8
GA     8
MT     8
OK     7
WY     7
SD     7
RI     7
NH     7
MA     7
AK     7
IN     7
IA     7
NV     6
LA     6
CA     6
NM     6
NE     6
AR     6
VA     6
CT     5
WI     5
MD     5
OH     5
UT     5
CO     5
TN     5
WA     4
ID     3
Name: State, dtype: int64
